<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

# LLM AS JUDGES untuk model  (Base & Fine Tune)

In [ ]:
%pip install opik openai comet_ml litellm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 885.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.2/565.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
import opik
from opik import Opik, track
from opik.evaluation import evaluate
from opik.evaluation.metrics import (Hallucination, AnswerRelevance, Moderation, Usefulness)
from opik.integrations.openai import track_openai
import openai
import os
from datetime import datetime
from getpass import getpass
import litellm

# Define project name to enable tracing
os.environ["OPIK_PROJECT_NAME"] = "Eval LLM NLP"

In [ ]:
from google.colab import userdata

In [ ]:
# opik configs | set di secret colab
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = userdata.get('OPIK_API_KEY')

In [ ]:
workspace = 'final-projek-nlp'

In [ ]:
opik.configure(workspace= workspace)

OPIK: Opik is already configured. You can check the settings by viewing the config file at /root/.opik.config


In [ ]:
client = Opik()

In [ ]:
os.environ['OPENROUTER_API_KEY'] = userdata.get('OPENROUTER_API_KEY')

LLM Client

In [ ]:
class LLMClient:
    def __init__(self, client_type: str = "openai", model: str = "gpt-4"):
        self.client_type = client_type
        self.model = model

        if self.client_type == "openai":
            self.client = track_openai(openai.OpenAI())
        else:  # LiteLLM via OpenRouter
            self.client = track_openai(
                openai.OpenAI(
                    base_url="https://openrouter.ai/api/v1",
                    api_key=os.environ["OPENROUTER_API_KEY"]
                )
            )

    def _get_litellm_response(self, query: str, system: str = "You are a helpful assistant."):
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": query}
        ]
        response = litellm.completion(model=self.model, messages=messages)
        return response.choices[0].message["content"]

    def _get_openai_response(self, query: str, system: str = "You are a helpful assistant.", **kwargs):
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": query}
        ]
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            **kwargs
        )
        return response.choices[0].message.content

    def query(self, query: str, system: str = "You are a helpful assistant.", **kwargs):
        if self.client_type == 'openai':
            return self._get_openai_response(query, system, **kwargs)
        else:
            return self._get_litellm_response(query, system)

In [ ]:
# To set clinet
MODEL= 'openrouter/deepseek/deepseek-r1' # model bebas milih selama ada di OpenRouter
llm_client = LLMClient(model=MODEL, client_type='litellm')

In [ ]:
llm_client.query('Pagi')

'Selamat pagi! Ada yang bisa saya bantu hari ini?'

list-dataset

[link dataset opik](https://www.comet.com/opik/final-projek-nlp/datasets)

In [ ]:
list_dataset = ['Model-1 Base | unsloth/mistral-7b-v0.3',
 'Model-1 ft | unsloth/mistral-7b-v0.3',
 'Model-2 Base | unsloth/mistral-7b-v0.3',
 'Model-2 ft | unsloth/mistral-7b-v0.3',
 'Model-3 Base | unsloth/mistral-7b-instruct-v0.2-bnb-4bit',
 'Model-3 ft | unsloth/mistral-7b-instruct-v0.2-bnb-4bit',
 'Model-4 Base | unsloth/llama-3-8b-bnb-4bit',
 'Model-4 ft | unsloth/llama-3-8b-bnb-4bit',
 'Model-5 Base | GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct',
 'Model-5 ft | GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct',
 'Model-6 Base | GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct',
 'Model-6 ft | GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct',
 'Model-7 Base | Yellow-AI-NLP/komodo-7b-base',
 'Model-7 ft | Yellow-AI-NLP/komodo-7b-base',
 'Model-8 Base | Yellow-AI-NLP/komodo-7b-base',
 'Model-8 ft | Yellow-AI-NLP/komodo-7b-base']

In [ ]:
# Create or get the dataset
# contoh disini karena untuk model 1
dataset = client.get_or_create_dataset(name="Model-1 Base | unsloth/mistral-7b-v0.3")

# Make custom model

In [ ]:
from opik.evaluation.models import OpikBaseModel
from typing import List, Dict, Any

class CustomModel(OpikBaseModel):
    def __init__(self, model_name: str = "gpt-4", client_type: str = "openai"):
        super().__init__(model_name)
        self.client = LLMClient(client_type=client_type, model=model_name)

    def generate_provider_response(self, messages: List[Dict[str, Any]], **kwargs: Any) -> str:
        if not messages or messages[-1]["role"] != "user":
            raise ValueError("Last message must be from user.")
        system_message = messages[0]["content"] if messages[0]["role"] == "system" else "You are a helpful assistant."
        user_message = messages[-1]["content"]
        return self.client.query(user_message, system=system_message, **kwargs)

    def generate_string(self, input: str, **kwargs: Any) -> str:
        return self.client.query(input, **kwargs)

In [ ]:
def simple_evaluation_task(item):
    return {
        "input": item["input"],
        "output": item["prediction"],
        "context": item["reference"]  # reference jadi context untuk hallucination check misal
    }

In [ ]:
# Model Custom untuk evaluasi
MODEL = 'openrouter/deepseek/deepseek-r1:free'
custom_model = CustomModel(model_name=MODEL, client_type="litellm")

In [ ]:
from opik.evaluation.metrics import (Hallucination, AnswerRelevance, Moderation, Usefulness)

In [ ]:
# Hallucination metric
hallucination_metric = Hallucination(model=custom_model)
AnswerRelevanceMetric = AnswerRelevance(model=custom_model)
ModerationMetric = Moderation(model=custom_model)
UsefulnessMetric = Usefulness(model=custom_model)

In [ ]:
# Nama eksperimen
experiment_name = f"{list_dataset[0]} | {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} | model as judges evaluation: {MODEL}"
experiment_name

'Model-1 Base | unsloth/mistral-7b-v0.3 | 2025-05-31_10-09-42 | model as judges evaluation: openrouter/deepseek/deepseek-r1:free'

In [ ]:
# Run evaluation
evaluation = evaluate(
    experiment_name=experiment_name,
    dataset=dataset,
    task=simple_evaluation_task,
    scoring_metrics=[hallucination_metric, AnswerRelevanceMetric, ModerationMetric, UsefulnessMetric],
    experiment_config={"model": MODEL,
                       "dataset" : dataset.name},
    task_threads=10,
)

Evaluation:   0%|          | 0/139 [00:00<?, ?it/s]

OPIK: Failed to parse model output: Failed to extract presumably JSON dictionary: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/opik/evaluation/metrics/llm_judges/parsing_helpers.py", line 22, in _extract_presumably_json_dict_or_raise
    return json.loads(json_string)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most rec


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric moderation_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '429 Too Many Requests' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

During handling of the above exc


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



...

OPIK: Failed to compute metric UsefulnessMetric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '429 Too Many Requests' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

During handling of the above exce


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric moderation_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '429 Too Many Requests' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/429

During handling of the above exc


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



KeyboardInterrupt: 

In [ ]:
# Nama eksperimen

MODEL = 'openrouter/openai/gpt-4o-mini'
custom_model = CustomModel(model_name=MODEL, client_type="litellm")

experiment_name = f"{list_dataset[0]} | {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} | model as judges evaluation: {MODEL}"
experiment_name

'Model-1 Base | unsloth/mistral-7b-v0.3 | 2025-05-31_03-42-25 | model as judges evaluation: openrouter/openai/gpt-4o-mini'

# Function for evaluation

In [ ]:
from datetime import datetime

def run_evaluation(model_name, dataset,no_dataset, thread, sample= None):
    custom_model = CustomModel(model_name=model_name, client_type="litellm")

    # Metrik evaluasi
    hallucination = Hallucination(model=custom_model)
    relevance = AnswerRelevance(model=custom_model)
    moderation = Moderation(model=custom_model)
    usefulness = Usefulness(model=custom_model)

    # Nama eksperimen
    experiment_name = f"{list_dataset[no_dataset]} | {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')} | model as judges evaluation: {model_name}"

    # Jalankan evaluasi
    evaluation = evaluate(
        experiment_name=experiment_name,
        dataset=dataset,
        task=simple_evaluation_task,
        scoring_metrics=[hallucination, relevance, moderation, usefulness],
        experiment_config={"model": model_name, "dataset": dataset.name},
        task_threads=thread,
        nb_samples=sample
    )

    return evaluation

# Model 1

## Model 1 Base

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 0 # index
task_thread = 4
sample = None
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

Evaluation:   0%|          | 0/139 [00:00<?, ?it/s]

╭─ Model-1 Base | unsloth/mistral-7b-v0.3 (139 samples) ─╮
│                                                        │
│ Total time:        00:04:49                            │
│ Number of samples: 139                                 │
│                                                        │
│ hallucination_metric: 0.7525 (avg)                     │
│ answer_relevance_metric: 0.4665 (avg)                  │
│ moderation_metric: 0.0230 (avg)                        │
│ UsefulnessMetric: 0.2561 (avg)                         │
│                                                        │
╰────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=422756;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=01972484-ad58-7f93-87b0-5c405db20ba5&dataset_id=019723cb-8132-7b6c-8a09-5e865fd216c6&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

## model  1 FT

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 1 # index
task_thread = 4
sample = None
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

Evaluation:   0%|          | 0/148 [00:00<?, ?it/s]

OPIK: Failed to compute metric UsefulnessMetric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



╭─ Model-1 ft | unsloth/mistral-7b-v0.3 (148 samples) ─╮
│                                                      │
│ Total time:        00:05:58                          │
│ Number of samples: 148                               │
│                                                      │
│ hallucination_metric: 0.5020 (avg)                   │
│ answer_relevance_metric: 0.8291 (avg)                │
│ moderation_metric: 0.0020 (avg)                      │
│ UsefulnessMetric: 0.6871 (avg) - 1 failed            │
│                                                      │
╰──────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=188937;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019724a7-da3c-7662-9d3f-882ddfb4b6ec&dataset_id=019723cb-82d2-751c-86cb-ef9e0213ec6b&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

# Model 2

## Model 2 Base

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 2 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-2 Base | unsloth/mistral-7b-v0.3


OPIK: Created a "Model-2 Base | unsloth/mistral-7b-v0.3" dataset at https://www.comet.com/opik/api/v1/session/redirect/datasets/?dataset_id=019724c2-c823-767c-bce4-7b5f7eefba53&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Evaluation: 0it [00:00, ?it/s]

╭─ Model-2 Base | unsloth/mistral-7b-v0.3 (0 samples) ─╮
│                                                      │
│ Total time:        00:00:00                          │
│ Number of samples: 0                                 │
│                                                      │
│                                                      │
╰──────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=233328;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019724c2-c8d4-76ec-87f8-16501f101225&dataset_id=019724c2-c823-767c-bce4-7b5f7eefba53&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

## model 2 ft

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 3 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-2 ft | unsloth/mistral-7b-v0.3


OPIK: Created a "Model-2 ft | unsloth/mistral-7b-v0.3" dataset at https://www.comet.com/opik/api/v1/session/redirect/datasets/?dataset_id=019724c3-29e4-7f7f-87a3-047609d2fe07&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Evaluation: 0it [00:00, ?it/s]

╭─ Model-2 ft | unsloth/mistral-7b-v0.3 (0 samples) ─╮
│                                                    │
│ Total time:        00:00:00                        │
│ Number of samples: 0                               │
│                                                    │
│                                                    │
╰────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=463521;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019724c3-2ad2-72e5-9878-146d034d9729&dataset_id=019724c3-29e4-7f7f-87a3-047609d2fe07&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

# Model 3

## model 3 base

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 4 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-3 Base | unsloth/mistral-7b-instruct-v0.2-bnb-4bit


Evaluation:   0%|          | 0/101 [00:00<?, ?it/s]

╭─ Model-3 Base | unsloth/mistral-7b-instruct-v0.2-bnb-4bit (101 samples) ─╮
│                                                                          │
│ Total time:        00:04:37                                              │
│ Number of samples: 101                                                   │
│                                                                          │
│ hallucination_metric: 0.7069 (avg)                                       │
│ answer_relevance_metric: 0.7718 (avg)                                    │
│ moderation_metric: 0.0010 (avg)                                          │
│ UsefulnessMetric: 0.6257 (avg)                                           │
│                                                                          │
╰──────────────────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=9135;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019724c3-cef9-75c3-be29-7dd504e2f6ef&dataset_id=019723cb-84c9-7023-8a99-5a1392ce82b3&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

## model 3 ft

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 5 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-3 ft | unsloth/mistral-7b-instruct-v0.2-bnb-4bit


Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

OPIK: Failed to compute metric UsefulnessMetric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



╭─ Model-3 ft | unsloth/mistral-7b-instruct-v0.2-bnb-4bit (100 samples) ─╮
│                                                                        │
│ Total time:        00:04:13                                            │
│ Number of samples: 100                                                 │
│                                                                        │
│ hallucination_metric: 0.4870 (avg)                                     │
│ answer_relevance_metric: 0.8265 (avg)                                  │
│ moderation_metric: 0.0080 (avg)                                        │
│ UsefulnessMetric: 0.7051 (avg) - 1 failed                              │
│                                                                        │
╰────────────────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=790136;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019724de-b534-7515-ae0a-3705d1a9d4d0&dataset_id=019723cb-865a-719c-9222-f56760c6134f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

# Model 4

## model 4 base

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 6 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-4 Base | unsloth/llama-3-8b-bnb-4bit


Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ Model-4 Base | unsloth/llama-3-8b-bnb-4bit (100 samples) ─╮
│                                                            │
│ Total time:        00:03:56                                │
│ Number of samples: 100                                     │
│                                                            │
│ hallucination_metric: 0.7510 (avg)                         │
│ answer_relevance_metric: 0.5640 (avg)                      │
│ moderation_metric: 0.0180 (avg)                            │
│ UsefulnessMetric: 0.3400 (avg)                             │
│                                                            │
╰────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=652055;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=01972504-012a-7e31-90e7-a2f204d35149&dataset_id=01972501-ff70-76e6-b842-1495d1994c85&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

## model 4 ft

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 7 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-4 ft | unsloth/llama-3-8b-bnb-4bit


Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

OPIK: Failed to compute metric UsefulnessMetric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



╭─ Model-4 ft | unsloth/llama-3-8b-bnb-4bit (100 samples) ─╮
│                                                          │
│ Total time:        00:03:50                              │
│ Number of samples: 100                                   │
│                                                          │
│ hallucination_metric: 0.4820 (avg)                       │
│ answer_relevance_metric: 0.8320 (avg)                    │
│ moderation_metric: 0.0050 (avg)                          │
│ UsefulnessMetric: 0.6990 (avg) - 1 failed                │
│                                                          │
╰──────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=533910;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019724ea-6705-713a-a63b-7111d64181be&dataset_id=019723cb-8908-7995-9572-4721a6ca30a4&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

# Model 5

## Model base 5

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 8 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-5 Base | GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct


Evaluation:   0%|          | 0/105 [00:00<?, ?it/s]

OPIK: Failed to compute metric UsefulnessMetric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric moderation_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



╭─ Model-5 Base | GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct (105 samples) ─╮
│                                                                                │
│ Total time:        00:05:47                                                    │
│ Number of samples: 105                                                         │
│                                                                                │
│ hallucination_metric: 0.4171 (avg)                                             │
│ answer_relevance_metric: 0.8543 (avg)                                          │
│ moderation_metric: 0.0019 (avg) - 1 failed                                     │
│ UsefulnessMetric: 0.7327 (avg) - 1 failed                                      │
│                                                                                │
╰────────────────────────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=41661;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=01972622-f198-76bf-80fc-690f05593d43&dataset_id=019723cb-8a7a-7716-8fec-61cb1bd84e64&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

## model 5 ft

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 9 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-5 ft | GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct


Evaluation:   0%|          | 0/103 [00:00<?, ?it/s]

OPIK: Failed to compute metric moderation_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric hallucination_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the ab

...

OPIK: Failed to compute metric answer_relevance_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



╭─ Model-5 ft | GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct (103 samples) ─╮
│                                                                              │
│ Total time:        00:05:24                                                  │
│ Number of samples: 103                                                       │
│                                                                              │
│ hallucination_metric: 0.4950 (avg) - 2 failed                                │
│ answer_relevance_metric: 0.8147 (avg) - 1 failed                             │
│ moderation_metric: 0.0000 (avg) - 1 failed                                   │
│ UsefulnessMetric: 0.6408 (avg)                                               │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=407861;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0197261d-8049-78af-a2a5-5e5c9cb3c4fa&dataset_id=019723cb-8c41-718a-9ef7-6dcc5c60607a&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

# Model 6

## model 6 base

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 10 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-6 Base | GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct


Evaluation:   0%|          | 0/102 [00:00<?, ?it/s]

OPIK: Failed to compute metric answer_relevance_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric answer_relevance_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the

...

OPIK: Failed to compute metric UsefulnessMetric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above 


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



╭─ Model-6 Base | GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct (102 samples) ─╮
│                                                                                │
│ Total time:        00:05:33                                                    │
│ Number of samples: 102                                                         │
│                                                                                │
│ hallucination_metric: 0.3284 (avg)                                             │
│ answer_relevance_metric: 0.8860 (avg) - 2 failed                               │
│ moderation_metric: 0.0000 (avg)                                                │
│ UsefulnessMetric: 0.7931 (avg) - 1 failed                                      │
│                                                                                │
╰────────────────────────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=588740;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0197262a-b93d-7897-a894-07e0be4103c2&dataset_id=019723cb-8d5b-7e96-8790-a198f338790d&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

## model ft 6

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 11 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-6 ft | GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct


Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

OPIK: Started logging traces to the "Eval LLM NLP" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=019725d1-bc0d-7dab-9718-c74f285b0fff&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


╭─ Model-6 ft | GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct (100 samples) ─╮
│                                                                              │
│ Total time:        00:05:26                                                  │
│ Number of samples: 100                                                       │
│                                                                              │
│ hallucination_metric: 0.5120 (avg)                                           │
│ answer_relevance_metric: 0.8505 (avg)                                        │
│ moderation_metric: 0.0120 (avg)                                              │
│ UsefulnessMetric: 0.7260 (avg)                                               │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=795130;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019725d1-b723-7739-812f-ad508a9453b5&dataset_id=019723cb-8e7a-72ec-901c-bb11d1f00abc&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

# Model 7

## Model base 7

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 12 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-7 Base | Yellow-AI-NLP/komodo-7b-base


Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

OPIK: Failed to compute metric moderation_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric answer_relevance_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



╭─ Model-7 Base | Yellow-AI-NLP/komodo-7b-base (104 samples) ─╮
│                                                             │
│ Total time:        00:06:39                                 │
│ Number of samples: 104                                      │
│                                                             │
│ hallucination_metric: 0.6365 (avg)                          │
│ answer_relevance_metric: 0.7218 (avg) - 1 failed            │
│ moderation_metric: 0.0136 (avg) - 1 failed                  │
│ UsefulnessMetric: 0.4904 (avg)                              │
│                                                             │
╰─────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=787460;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019725ed-58bc-7000-bb90-bae78df71150&dataset_id=019723cb-8fa0-7fbc-9dfc-669606ac425f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

## model ft 7

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 13 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-7 ft | Yellow-AI-NLP/komodo-7b-base


Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ Model-7 ft | Yellow-AI-NLP/komodo-7b-base (100 samples) ─╮
│                                                           │
│ Total time:        00:06:33                               │
│ Number of samples: 100                                    │
│                                                           │
│ hallucination_metric: 0.5770 (avg)                        │
│ answer_relevance_metric: 0.8215 (avg)                     │
│ moderation_metric: 0.0070 (avg)                           │
│ UsefulnessMetric: 0.6800 (avg)                            │
│                                                           │
╰───────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=933261;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019725f3-796f-7188-8868-31088821ec52&dataset_id=019723cb-90fc-7ec4-9fad-17986ad09b88&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

# Model 8

## Model 8 base

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 14 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-8 Base | Yellow-AI-NLP/komodo-7b-base


Evaluation:   0%|          | 0/104 [00:00<?, ?it/s]

OPIK: Failed to compute metric moderation_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



╭─ Model-8 Base | Yellow-AI-NLP/komodo-7b-base (104 samples) ─╮
│                                                             │
│ Total time:        00:06:50                                 │
│ Number of samples: 104                                      │
│                                                             │
│ hallucination_metric: 0.6433 (avg)                          │
│ answer_relevance_metric: 0.7250 (avg)                       │
│ moderation_metric: 0.0117 (avg) - 1 failed                  │
│ UsefulnessMetric: 0.4788 (avg)                              │
│                                                             │
╰─────────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=56913;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019725ff-7919-7927-8a92-616102595cac&dataset_id=019723cb-9236-75e2-9f5b-89363510da4d&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

## Model ft 8

In [ ]:
MODEL = 'openrouter/openai/gpt-4o-mini'
dataset_no = 15 # index
task_thread = 4
sample = None
print('model:', list_dataset[dataset_no])
dataset = client.get_or_create_dataset(name=list_dataset[dataset_no])
result = run_evaluation(MODEL, dataset, dataset_no, task_thread, sample=sample)

model: Model-8 ft | Yellow-AI-NLP/komodo-7b-base


Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

OPIK: Failed to compute metric answer_relevance_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric moderation_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the above


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



OPIK: Failed to compute metric hallucination_metric. Score result will be marked as failed.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 159, in _make_common_sync_call
    response = sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 694, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 676, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Server error '500 Internal Server Error' for url 'https://openrouter.ai/api/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500

During handling of the ab

╭─ Model-8 ft | Yellow-AI-NLP/komodo-7b-base (100 samples) ─╮
│                                                           │
│ Total time:        00:05:31                               │
│ Number of samples: 100                                    │
│                                                           │
│ hallucination_metric: 0.4667 (avg) - 1 failed             │
│ answer_relevance_metric: 0.8318 (avg) - 1 failed          │
│ moderation_metric: 0.0020 (avg) - 1 failed                │
│ UsefulnessMetric: 0.7080 (avg)                            │
│                                                           │
╰───────────────────────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=575827;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0197260f-9417-72c9-9c36-dc9a732f4105&dataset_id=019723cb-9366-7982-93cb-b4cdf0119083&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.